**Introduction**
  We propose to achieve movie genre classification based only on movie poster images. A deep neural network is constructed to jointly describe visual appearance and object information, and classify a given movie poster image into genres. Because a movie may belong to multiple genres, this is a multi-label image classification problem. To facilitate related studies, we collect a large-scale movie poster dataset, associated with various metadata. Based on this dataset, we fine-tune a pre-trained convolutional neural network to extract visual representation and adopt a state-of-the-art framework to detect objects in posters. Two types of information are then integrated by the proposed neural network. In the evaluation, we show that the proposed method yields encouraging performance, which is much better than previous works. "

In [13]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 61kB/s 
     |████████████████████████████████| 3.8MB 38.8MB/s 
     |████████████████████████████████| 450kB 48.8MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [0]:
import pandas as pd
import json
import glob  
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D


In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [0]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [21]:
print(tf.__version__)

2.0.0


In [52]:
!git clone https://github.com/sweetcoonoor/Multiclassification_Genre.git

fatal: destination path 'Multiclassification_Genre' already exists and is not an empty directory.


In [60]:
!git clone https://github.com/laxmimerit/Movies-Poster_Dataset.git

Cloning into 'Movies-Poster_Dataset'...
remote: Enumerating objects: 7873, done.
remote: Counting objects: 100% (7873/7873), done.
remote: Compressing objects: 100% (7873/7873), done.
remote: Total 7873 (delta 1), reused 7872 (delta 0), pack-reused 0
Receiving objects: 100% (7873/7873), 246.59 MiB | 38.64 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (7871/7871), done.


In [0]:
ff=open("1980.txt","r")
Movies=ff.read()
comma=Movies.replace("}","},")
newc=comma[:-2]
sqaureAdd="["+newc+"]"
aaa=sqaureAdd.replace("\n","")
jj=aaa.replace("ObjectId(","").replace("\")","\"")
fin=json.loads(jj)
df = pd.DataFrame.from_dict(fin, orient='columns')

In [0]:
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
cols=df.columns
dfFin=pd.DataFrame(columns=cols)
dfFin
dfFin2=dfFin

In [0]:
all_files = glob.glob("*.txt")

print((all_files))
count=0
for files in all_files:
  try:
    ff=open(files,"r",encoding='utf-16')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
  except:
    ff=open(files,"r",encoding='utf-8')
    stringg=ff.read()
    comma=Movies.replace("}","},")
    newc=comma[:-2]
    sqaureAdd="["+newc+"]"
    aaa=sqaureAdd.replace("\n","")
    jj=aaa.replace("ObjectId(","").replace("\")","\"")
    fin=json.loads(jj)
    df = pd.DataFrame.from_dict(fin, orient='columns')
    dfFin=pd.concat([dfFin,df])
    

['2005.txt', '1980.txt', '2002.txt', '1985.txt', '2015.txt', '1989.txt', '2013.txt', '2004.txt', '2014.txt', '1987.txt', '2007.txt', '1990.txt', '1988.txt', '1997.txt', '1993.txt', '1982.txt', '2010.txt', '2011.txt', '1981.txt', '1983.txt', '1999.txt', '1998.txt', '1996.txt', '1995.txt', '2009.txt', '2003.txt', '2012.txt', '1984.txt', '1991.txt', '1992.txt', '2001.txt', '2000.txt', '1994.txt', '2008.txt', '1986.txt', '2006.txt']


In [26]:
Movie_data=pd.concat([dfFin,dfFin2])
#Movie_data.head()
Movie_data.shape


(3492, 20)

Data Exploration and Pre-processing

In [33]:
Movie_Final=Movie_data[['imdbID','Title','Genre',]]
Movie_Final.shape

(3492, 3)

In [0]:
Movie_Final.to_csv('Movie_genre.csv',encoding="ISO-8859-1")




In [0]:
#Onehot encoding
df = pd.read_csv("Movie_genre.csv")

df.dropna(subset=['Genre'],inplace=True)
df.drop_duplicates(subset=['imdbID'], keep='first', inplace=True)
df.set_index('imdbID',inplace=True)
target = df['Genre'].apply(lambda x: [w.strip() for w in x.split(',')])
genres = pd.get_dummies(target.apply(pd.Series).stack()).sum(level=0)


In [67]:
df

,Unnamed: 0,Title,Genre
imdbID,,,
tt0080684,0,Star Wars: Episode V - The Empire Strikes Back,"Action, Adventure, Fantasy"
tt0081562,1,Stir Crazy,"Comedy, Crime"
tt0080339,2,Airplane!,Comedy
tt0080377,3,Any Which Way You Can,"Action, Comedy"
tt0081375,4,Private Benjamin,"Comedy, War"
tt0080549,5,Coal Miner's Daughter,"Biography, Drama, Music"
tt0081529,6,Smokey and the Bandit II,"Action, Comedy"
tt0080453,7,The Blue Lagoon,"Adventure, Drama, Romance"
tt0080455,8,The Blues Brothers,"Action, Comedy, Crime"


In [0]:

target.shape
Movie_Final

In [71]:
image_width=250
image_height=250
X=[]
for i in tqdm(range(Movie_Final.shape[0])):
  path='/content/Movies-Poster_Dataset/Images' +Movie_Final['imdbID'][i] +'*.jpg'
  img=image.load_img(path,target_size=(image_width,image_height,3))
  img=image.img_to_array(img)
  img=img/255.0
  X.append(img)
              
x=np.array(x)

  0%|          | 0/3492 [00:00<?, ?it/s]


AttributeError: ignored

In [72]:
for i in tqdm(range(data.shape[0])):
  path = '/content/Movies-Poster_Dataset/Images/' + data['Id'][i] + '.jpg'
  img = image.load_img(path, target_size=(img_width, img_height, 3))
  img = image.img_to_array(img)
  img = img/255.0
  X.append(img)

NameError: ignored